In [1]:
!pip install -q opencv-python-headless matplotlib seaborn sahi==0.11.18 ultralytics albumentations

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 29.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.7 MB/s eta 0:00:0

In [5]:
import cv2
import os
import random
import shutil
import xml.etree.ElementTree as ET
from glob import glob
import albumentations as A
from sahi import AutoDetectionModel
from sahi.predict import get_prediction, get_sliced_prediction, predict
from ultralytics import YOLO
from IPython.display import Image, display
from sahi.utils.yolov8 import download_yolov8n_model, download_yolov8m_model, download_yolov8l_model, download_yolov8x_model

/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.7' (you have '2.0.4'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [3]:
# --- 1. Thông số ---
INPUT_IMAGE_DIR = "/kaggle/input/yellostickytrap/yellow-sticky-traps-dataset/images"
INPUT_ANNOTATION_DIR = "/kaggle/input/yellostickytrap/yellow-sticky-traps-dataset/annotations"
OUTPUT_DIR = "/kaggle/working/yolo_dataset"
CLASSES = {'MR': 0, 'NC': 1, 'WF': 2}
TRAIN_RATIO, VAL_RATIO, TEST_RATIO = 0.7, 0.15, 0.15
SEED = 42

# --- 2. Tạo thư mục ---
for split in ['train', 'val', 'test']:
    os.makedirs(f"{OUTPUT_DIR}/images/{split}", exist_ok=True)
    os.makedirs(f"{OUTPUT_DIR}/labels/{split}", exist_ok=True)

# --- 3. Hàm chuyển annotation XML -> YOLO format ---
def convert_annotation(xml_path, w, h):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    labels = []
    for obj in root.findall('object'):
        cls_name = obj.find('name').text
        if cls_name not in CLASSES:
            continue
        cls_id = CLASSES[cls_name]
        bnd = obj.find('bndbox')
        xmin, ymin = int(bnd.find('xmin').text), int(bnd.find('ymin').text)
        xmax, ymax = int(bnd.find('xmax').text), int(bnd.find('ymax').text)
        xc = ((xmin + xmax) / 2) / w
        yc = ((ymin + ymax) / 2) / h
        bw = (xmax - xmin) / w
        bh = (ymax - ymin) / h
        labels.append([cls_id, xc, yc, bw, bh])
    return labels

# --- 4. Augmentation (chỉ áp dụng cho train) ---
augment = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=1.0)
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

# --- 5. Chia tập ---
image_paths = sorted(glob(f"{INPUT_IMAGE_DIR}/*.jpg"))
random.seed(SEED)
random.shuffle(image_paths)

n = len(image_paths)
train_end = int(n * TRAIN_RATIO)
val_end = train_end + int(n * VAL_RATIO)
splits = {
    'train': image_paths[:train_end],
    'val': image_paths[train_end:val_end],
    'test': image_paths[val_end:]
}

# --- 6. Xử lý ảnh theo từng split ---
for split, paths in splits.items():
    for img_path in paths:
        name = os.path.basename(img_path)
        xml_path = os.path.join(INPUT_ANNOTATION_DIR, name.replace('.jpg', '.xml'))
        img = cv2.imread(img_path)
        if img is None or not os.path.exists(xml_path):
            continue
        h, w = img.shape[:2]
        bboxes = convert_annotation(xml_path, w, h)
        if not bboxes:
            continue

        # Nếu là train: áp dụng augment 2 lần (gốc + augment)
        num_augs = 2 if split == 'train' else 1

        for i in range(num_augs):
            # Nếu augment, thì thực hiện augment. Nếu i == 0 thì giữ nguyên ảnh gốc
            if split == 'train' and i > 0:
                class_labels = [b[0] for b in bboxes]
                bbox_only = [b[1:] for b in bboxes]
                transformed = augment(image=img, bboxes=bbox_only, class_labels=class_labels)
                transformed_img = transformed['image']
                transformed_bboxes = transformed['bboxes']
                transformed_labels = transformed['class_labels']
            else:
                transformed_img = img
                transformed_bboxes = [b[1:] for b in bboxes]
                transformed_labels = [b[0] for b in bboxes]

            if not transformed_bboxes:
                continue

            # Ghi ảnh và nhãn
            out_name = name.replace('.jpg', f'_{i}.jpg') if i > 0 else name
            out_txt = out_name.replace('.jpg', '.txt')
            cv2.imwrite(f"{OUTPUT_DIR}/images/{split}/{out_name}", transformed_img)
            with open(f"{OUTPUT_DIR}/labels/{split}/{out_txt}", 'w') as f:
                for cid, (x, y, w_, h_) in zip(transformed_labels, transformed_bboxes):
                    f.write(f"{cid} {x:.6f} {y:.6f} {w_:.6f} {h_:.6f}\n")

print("✅ Dataset đã được chia và augmented (chỉ train) xong.")

# --- 7. Tạo file data.yaml ---
data_yaml = f'''
train: {OUTPUT_DIR}/images/train
val: {OUTPUT_DIR}/images/val
test: {OUTPUT_DIR}/images/test

nc: {len(CLASSES)}
names: {list(CLASSES.keys())}
'''
with open("/kaggle/working/data.yaml", "w") as f:
    f.write(data_yaml)

print("✅ Đã tạo file data.yaml")


✅ Dataset đã được chia và augmented (chỉ train) xong.
✅ Đã tạo file data.yaml


# YOLOv8n

In [5]:
model = YOLO('yolov8n.pt')

model.train(
    data='/kaggle/working/data.yaml',
    epochs=250,
    imgsz=640,
    batch=32,  
    device='cuda',
    project='/kaggle/working',
    name='yolov8n_insects',
    pretrained=True,
    optimizer='AdamW',
    lr0=0.001,
    lrf=0.01,
    warmup_epochs=3,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=0.0,           # không xoay gây mất object
    translate=0.0,         # không dịch
    scale=0.5,
    shear=0.0,
    perspective=0.0,
    mosaic=1.0,            # giữ mặc định (tốt cho small object)
    mixup=0.0,             # tắt
    patience=50,
    close_mosaic=15        # sau 15 epoch tắt mosaic để fine-tune
)


Ultralytics 8.3.138 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=15, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=250, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8n_insects, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=50, perspective=0.0, plots=True, pose=12.0, pret

train: Scanning /kaggle/working/yolo_dataset/labels/train.cache... 396 images, 0 backgrounds, 0 corrupt: 100%|██████████| 396/396 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2547.8±2231.8 MB/s, size: 2007.3 KB)


val: Scanning /kaggle/working/yolo_dataset/labels/val.cache... 42 images, 0 backgrounds, 0 corrupt: 100%|██████████| 42/42 [00:00<?, ?it/s]


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.71it/s]


                   all         42       1112      0.848      0.775      0.839      0.378
                    MR         25        234      0.851      0.863      0.869      0.374
                    NC         15        133      0.928      0.962      0.984      0.504
                    WF         24        745      0.765      0.501      0.663      0.256


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.1ms preprocess, 1.3ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to /kaggle/working/yolov8n_insects


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ce1f1825b90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04

In [6]:
# Load mô hình tốt nhất đã train xong
model = YOLO('/kaggle/working/yolov8n_insects/weights/best.pt')

# Đánh giá trên tập test (được chỉ định trong data.yaml)
metrics = model.val(
    data='/kaggle/working/data.yaml',
    split='test',
    save=True,         # lưu ảnh + kết quả
    save_json=True     # lưu file predictions.json để đánh giá mAP
)

Ultralytics 8.3.138 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2601.4±1323.7 MB/s, size: 2322.9 KB)


val: Scanning /kaggle/working/yolo_dataset/labels/test... 44 images, 0 backgrounds, 0 corrupt: 100%|██████████| 44/44 [00:00<00:00, 1391.22it/s]

val: New cache created: /kaggle/working/yolo_dataset/labels/test.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:09<00:00,  3.01s/it]


                   all         44        967      0.824      0.748      0.796      0.361
                    MR         30        334      0.839      0.856      0.873      0.371
                    NC         19        120      0.927      0.845       0.92      0.491
                    WF         24        513      0.707      0.544      0.595      0.221


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 5.7ms preprocess, 7.3ms inference, 0.0ms loss, 3.2ms postprocess per image
Saving runs/detect/val/predictions.json...
Results saved to runs/detect/val


In [25]:
# --- Phân tích thêm ---
output_json = "/kaggle/working/runs/detect/val/predictions.json"
with open(output_json) as f:
    preds = json.load(f)

print("\n--- Thống kê ---")
print(f"Số bbox ground truth : {len(gt.dataset['annotations'])}")
print(f"Số bbox prediction   : {len(preds)}")

# In danh sách category (để kiểm tra khớp)
print("\n--- Danh sách categories ---")
for cat in gt.loadCats(gt.getCatIds()):
    print(f"ID: {cat['id']}  |  Name: {cat['name']}")


--- Thống kê ---
Số bbox ground truth : 967
Số bbox prediction   : 3817

--- Danh sách categories ---
ID: 0  |  Name: MR
ID: 1  |  Name: NC
ID: 2  |  Name: WF


# KẾT HỢP YOLOV8N VỚI SAHI

In [18]:
import os
import json
from ultralytics import YOLO
from sahi import AutoDetectionModel
from sahi.predict import get_sliced_prediction

# --- Cấu hình ---
test_images_dir = "/kaggle/working/yolo_dataset/images/test"
model_path = "/kaggle/working/yolov8n_insects/weights/best.pt"
gt_coco_path = "/kaggle/input/eval_sahi/pytorch/default/1/eval/instances_test_gt.json"
output_json_path = "/kaggle/working/sahi_test_labels.json"

# SAHI slicing & postprocessing
slice_height = 128
slice_width = 128
overlap_height_ratio = 0.35
overlap_width_ratio = 0.35
perform_standard_pred = True
postprocess_type = "NMS"
postprocess_match_metric = "IOU"
postprocess_match_threshold = 0.45
postprocess_class_agnostic = False
verbose = 1
auto_slice_resolution = False
confidence_threshold = 0.5
device = "cuda"

# --- Load ground truth COCO ---
with open(gt_coco_path, 'r') as f:
    gt_coco = json.load(f)

filename_to_id = {img['file_name']: img['id'] for img in gt_coco['images']}
valid_category_ids = {cat['id'] for cat in gt_coco['categories']}
id2name = {cat['id']: cat['name'] for cat in gt_coco['categories']}  # ✅ ánh xạ ID → tên nhãn

# --- Load YOLOv8 model ---
yolo_model = YOLO(model_path)
detection_model = AutoDetectionModel.from_pretrained(
    model_type='yolov8',
    model=yolo_model,
    confidence_threshold=confidence_threshold,
    device=device,
)

# --- Inference và lưu kết quả có tên nhãn ---
coco_predictions = []

image_filenames = sorted([
    f for f in os.listdir(test_images_dir)
    if f.lower().endswith(('.jpg', '.jpeg', '.png'))
])

for img_name in image_filenames:
    image_path = os.path.join(test_images_dir, img_name)
    print(f"🔍 Processing: {img_name}")

    result = get_sliced_prediction(
        image=image_path,
        detection_model=detection_model,
        slice_height=slice_height,
        slice_width=slice_width,
        overlap_height_ratio=overlap_height_ratio,
        overlap_width_ratio=overlap_width_ratio,
        perform_standard_pred=perform_standard_pred,
        postprocess_type=postprocess_type,
        postprocess_match_metric=postprocess_match_metric,
        postprocess_match_threshold=postprocess_match_threshold,
        postprocess_class_agnostic=postprocess_class_agnostic,
        verbose=verbose,
        auto_slice_resolution=auto_slice_resolution,
    )

    image_id = filename_to_id.get(img_name)
    if image_id is None:
        print(f"⚠️ Không tìm thấy image_id cho {img_name}, bỏ qua.")
        continue

    # ✅ Duyệt từng object_prediction và ghi kèm tên nhãn
    for obj_pred in result.object_prediction_list:
        category_id = obj_pred.category.id
        category_name = obj_pred.category.name
        if category_id not in valid_category_ids:
            print(f"⚠️ category_id {category_id} không hợp lệ, bỏ qua.")
            continue

        x1, y1, x2, y2 = obj_pred.bbox.to_xyxy()
        bbox = [x1, y1, x2 - x1, y2 - y1]  # COCO format: x, y, w, h

        coco_predictions.append({
            "image_id": image_id,
            "category_id": category_id,
            "category_name": category_name,  # ✅ thêm tên nhãn
            "bbox": [float(round(v, 2)) for v in bbox],
            "score": float(round(obj_pred.score.value, 4))
        })

# --- Ghi kết quả ra file JSON ---
with open(output_json_path, "w") as f:
    json.dump(coco_predictions, f, indent=2)

print(f"\n✅ SAHI inference hoàn tất. Kết quả với nhãn lưu tại: {output_json_path}")


🔍 Processing: 1003.jpg
Performing prediction on 2542 slices.
🔍 Processing: 1011.jpg
Performing prediction on 2542 slices.
🔍 Processing: 1012.jpg
Performing prediction on 2542 slices.
🔍 Processing: 1013.jpg
Performing prediction on 2542 slices.
🔍 Processing: 1015.jpg
Performing prediction on 2542 slices.
🔍 Processing: 1016.jpg
Performing prediction on 2542 slices.
🔍 Processing: 1023.jpg
Performing prediction on 2542 slices.
🔍 Processing: 1024.jpg
Performing prediction on 2542 slices.
🔍 Processing: 1026.jpg
Performing prediction on 2542 slices.
🔍 Processing: 1031.jpg
Performing prediction on 2542 slices.
🔍 Processing: 1044.jpg
Performing prediction on 2542 slices.
🔍 Processing: 1047.jpg
Performing prediction on 2542 slices.
🔍 Processing: 1052.jpg
Performing prediction on 2542 slices.
🔍 Processing: 1057.jpg
Performing prediction on 2542 slices.
🔍 Processing: 1067.jpg
Performing prediction on 2542 slices.
🔍 Processing: 1071.jpg
Performing prediction on 2542 slices.
🔍 Processing: 1079.jpg
P

In [19]:
# --- Phân tích thêm ---
with open(output_json_path) as f:
    preds = json.load(f)

print("\n--- Thống kê ---")
print(f"Số bbox ground truth : {len(gt.dataset['annotations'])}")
print(f"Số bbox prediction   : {len(preds)}")

# In danh sách category (để kiểm tra khớp)
print("\n--- Danh sách categories ---")
for cat in gt.loadCats(gt.getCatIds()):
    print(f"ID: {cat['id']}  |  Name: {cat['name']}")


--- Thống kê ---
Số bbox ground truth : 967
Số bbox prediction   : 699

--- Danh sách categories ---
ID: 0  |  Name: MR
ID: 1  |  Name: NC
ID: 2  |  Name: WF


In [27]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import json

# === Đọc Ground Truth và Prediction ===
gt_coco_path = "/kaggle/input/eval_sahi/pytorch/default/1/eval/instances_test_gt.json"
pred_coco_path = "/kaggle/working/sahi_test_labels.json"

gt = COCO(gt_coco_path)
dt = gt.loadRes(pred_coco_path)

# === Đánh giá mAP theo chuẩn COCO ===
coco_eval = COCOeval(gt, dt, iouType='bbox')
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()

# === Lấy các chỉ số cần ===
map_50_95 = coco_eval.stats[0]  # mAP@[0.5:0.95]
map_50 = coco_eval.stats[1]     # mAP@0.5
ar_100 = coco_eval.stats[8]     # AR@100

# === Tính Precision / Recall THỦ CÔNG theo IoU >= 0.5 ===
# Cách: lặp qua từng ảnh, so sánh bbox prediction vs ground truth → tính TP / FP / FN

import numpy as np

def compute_tp_fp_fn(gt: COCO, preds, iou_threshold=0.5):
    image_ids = gt.getImgIds()
    tp = 0
    fp = 0
    fn = 0

    for image_id in image_ids:
        gt_ann_ids = gt.getAnnIds(imgIds=image_id)
        gt_anns = gt.loadAnns(gt_ann_ids)
        gt_boxes = [ann['bbox'] for ann in gt_anns]
        gt_used = [False] * len(gt_boxes)

        # Prediction bboxes cho ảnh này
        pred_boxes = [ann['bbox'] for ann in preds if ann['image_id'] == image_id]
        pred_scores = [ann['score'] for ann in preds if ann['image_id'] == image_id]

        for pb in pred_boxes:
            matched = False
            for i, gb in enumerate(gt_boxes):
                if gt_used[i]:
                    continue
                iou = compute_iou(pb, gb)
                if iou >= iou_threshold:
                    tp += 1
                    gt_used[i] = True
                    matched = True
                    break
            if not matched:
                fp += 1

        # Số bbox GT còn lại chưa match là FN
        fn += gt_used.count(False)

    return tp, fp, fn

def compute_iou(boxA, boxB):
    # box = [x, y, w, h]
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[0] + boxA[2], boxB[0] + boxB[2])
    yB = min(boxA[1] + boxA[3], boxB[1] + boxB[3])

    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = boxA[2] * boxA[3]
    boxBArea = boxB[2] * boxB[3]
    iou = interArea / float(boxAArea + boxBArea - interArea + 1e-6)
    return iou

# === Load lại prediction để xử lý dễ hơn ===
with open(pred_coco_path) as f:
    predictions = json.load(f)

tp, fp, fn = compute_tp_fp_fn(gt, predictions, iou_threshold=0.5)

precision = tp / (tp + fp + 1e-6)
recall = tp / (tp + fn + 1e-6)

# === In kết quả ===
print("\n--- 📈 Metrics Summary ---")
print(f"mAP@0.5              : {map_50:.4f}")
print(f"mAP@0.5:0.95         : {map_50_95:.4f}")
# print(f"Recall (AR@100)      : {ar_100:.4f}")
print(f"Precision     : {precision:.4f}")
print(f"Recall        : {recall:.4f}")
print(f"TP = {tp} | FP = {fp} | FN = {fn}")


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.35s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.252
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.605
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.157
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.008
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.107
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.359
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.049
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.267
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets